In [ ]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import os

In [ ]:
import pandas as pd
import numpy as np
import os
import numpy as np
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
%matplotlib inline
import seaborn as sns
import statsmodels.formula.api as smf
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

In [ ]:
path = '../output/'

In [ ]:
joined_data = pd.read_csv(path + 'data_btu_railbelt.csv')
joined_data = joined_data.drop(columns =['Unnamed: 0'])

In [ ]:
joined_data.head()

# even split between fairbanks and anchorage--downsample anchorage

In [ ]:
# Fairbanks
zip_fb = [99701, 99702, 99703, 99705, 99706, 99707, 99708, 99709, 99710, 
          99711, 99712, 99714, 99716, 99725, 99775, 99790]

# Anchorage
zip_an = [99501, 99502, 99503, 99504, 99505, 99506, 99507, 99508, 99509, 99510,
           99511, 99513, 99514, 99515, 99516, 99517, 99518, 99519, 99520, 99521,
           99522, 99523, 99524, 99529, 99530, 99540, 99599, 99695]

In [ ]:
fairbanks_only = joined_data[joined_data['zip_code'].isin(zip_fb)]
anchorage_only = joined_data[joined_data['zip_code'].isin(zip_an)]

In [ ]:
anchorage_ind_keep = np.random.choice(len(anchorage_only), len(fairbanks_only)).tolist()
new_anchorage = anchorage_only.iloc[anchorage_ind_keep, :]

In [ ]:
# combined data with augemented fairbanks data
combined_fair_anch = new_anchorage.copy(deep=True)
combined_fair_anch = combined_fair_anch.append(fairbanks_only)

In [ ]:
# change FD cols to be positive
combined_fair_anch['FD_1981_2010'] = abs(combined_fair_anch['FD_1981_2010'])
combined_fair_anch['FD_2000'] = abs(combined_fair_anch['FD_2000'])
combined_fair_anch['FD_2010'] = abs(combined_fair_anch['FD_2010'])
combined_fair_anch['FD_90'] = abs(combined_fair_anch['FD_90'])
combined_fair_anch['FD_80'] = abs(combined_fair_anch['FD_80'])

In [ ]:
Xy_fairbanks = combined_fair_anch[combined_fair_anch['zip_code'].isin(zip_fb)]
Xy_anchorage = combined_fair_anch[combined_fair_anch['zip_code'].isin(zip_an)]

In [ ]:
# Create numpy arrays for scikit learn 
features_dropped = ['annual_btu_persqft', 'annual_btu', 'age', 'osm_id', 'zip_code', 'zip_group']
# features_kept = combined_fair_anch.drop(features_dropped, axis=1).columns

X_fair = np.array(Xy_fairbanks.drop(features_dropped, axis=1))
y_fair = np.array(Xy_fairbanks['annual_btu_persqft'])
y_fair = y_fair.reshape([y_fair.shape[0],1])

X_anch = np.array(Xy_anchorage.drop(features_dropped, axis=1))
y_anch = np.array(Xy_anchorage['annual_btu_persqft'])
y_anch = y_anch.reshape([y_anch.shape[0],1])

In [ ]:
# standardize data

scaler = StandardScaler()
scaler.fit(np.vstack((X_fair, X_anch)))
X_fair = scaler.transform(X_fair)
X_anch = scaler.transform(X_anch)

scaler = StandardScaler()
scaler.fit(np.vstack((y_fair, y_anch)))
y_fair = scaler.transform(y_fair)
y_anch = scaler.transform(y_anch)



In [ ]:
# Divide intro training and testing datasets, with a 30-70 ratio
X_train_anch, X_test_anch, y_train_anch, y_test_anch = train_test_split(X_anch, y_anch, test_size = 0.3, random_state = 1)

X_train_fair, X_test_fair, y_train_fair, y_test_fair = train_test_split(X_fair, y_fair, test_size = 0.3, random_state = 1)



In [ ]:
# combine fairbanks and anchorage
X_train = np.vstack((X_train_anch, X_train_fair))
X_test = np.vstack((X_test_anch, X_test_fair))
y_train = np.vstack((y_train_anch, y_train_fair))
y_test = np.vstack((y_test_anch, y_test_fair))

In [ ]:
with open(path + 'data_sampling/equal_anch_fair/x_train_downsample_anchorage.npy', 'wb') as f:
    np.save(f, X_train)

    
with open(path + 'data_sampling/equal_anch_fair/x_test_downsample_anchorage.npy', 'wb') as f:
    np.save(f, X_test)
    
    
with open(path + 'data_sampling/equal_anch_fair/y_train_downsample_anchorage.npy', 'wb') as f:
    np.save(f, y_train)
    
with open(path + 'data_sampling/equal_anch_fair/y_test_downsample_anchorage.npy', 'wb') as f:
    np.save(f, y_test)

# even split between fairbanks and anchorage--upsample fairbanks

In [ ]:
# Fairbanks
zip_fb = [99701, 99702, 99703, 99705, 99706, 99707, 99708, 99709, 99710, 
          99711, 99712, 99714, 99716, 99725, 99775, 99790]

# Anchorage
zip_an = [99501, 99502, 99503, 99504, 99505, 99506, 99507, 99508, 99509, 99510,
           99511, 99513, 99514, 99515, 99516, 99517, 99518, 99519, 99520, 99521,
           99522, 99523, 99524, 99529, 99530, 99540, 99599, 99695]

In [ ]:
fairbanks_only = joined_data[joined_data['zip_code'].isin(zip_fb)]
anchorage_only = joined_data[joined_data['zip_code'].isin(zip_an)]

In [ ]:
new_fairbanks = fairbanks_only.copy(deep=True)
num_samples = len(anchorage_only) - len(fairbanks_only)
new_fairbanks = new_fairbanks.append(fairbanks_only.sample(n = num_samples, replace = True))

In [ ]:
new_fairbanks.head()

In [ ]:
# combined data with augemented fairbanks data
combined_fair_anch = new_fairbanks.copy(deep=True)
combined_fair_anch = combined_fair_anch.append(anchorage_only)

In [ ]:
combined_fair_anch.head()

In [ ]:
combined_fair_anch.head()

In [ ]:
combined_fair_anch.columns

In [ ]:
# change FD cols to be positive
combined_fair_anch['FD_1981_2010'] = abs(combined_fair_anch['FD_1981_2010'])
combined_fair_anch['FD_2000'] = abs(combined_fair_anch['FD_2000'])
combined_fair_anch['FD_2010'] = abs(combined_fair_anch['FD_2010'])
combined_fair_anch['FD_90'] = abs(combined_fair_anch['FD_90'])
combined_fair_anch['FD_80'] = abs(combined_fair_anch['FD_80'])

In [ ]:
Xy_fairbanks = combined_fair_anch[combined_fair_anch['zip_code'].isin(zip_fb)]
Xy_anchorage = combined_fair_anch[combined_fair_anch['zip_code'].isin(zip_an)]

In [ ]:
# Create numpy arrays for scikit learn 
features_dropped = ['annual_btu_persqft', 'annual_btu', 'age', 'osm_id', 'zip_code', 'zip_group']
# features_kept = combined_fair_anch.drop(features_dropped, axis=1).columns

X_fair = np.array(Xy_fairbanks.drop(features_dropped, axis=1))
y_fair = np.array(Xy_fairbanks['annual_btu_persqft'])
y_fair = y_fair.reshape([y_fair.shape[0],1])

X_anch = np.array(Xy_anchorage.drop(features_dropped, axis=1))
y_anch = np.array(Xy_anchorage['annual_btu_persqft'])
y_anch = y_anch.reshape([y_anch.shape[0],1])

In [ ]:
# standardize data

scaler = StandardScaler()
scaler.fit(np.vstack((X_fair, X_anch)))
X_fair = scaler.transform(X_fair)
X_anch = scaler.transform(X_anch)

scaler = StandardScaler()
scaler.fit(np.vstack((y_fair, y_anch)))
y_fair = scaler.transform(y_fair)
y_anch = scaler.transform(y_anch)



In [ ]:
# Divide intro training and testing datasets, with a 30-70 ratio
X_train_anch, X_test_anch, y_train_anch, y_test_anch = train_test_split(X_anch, y_anch, test_size = 0.3, random_state = 1)

X_train_fair, X_test_fair, y_train_fair, y_test_fair = train_test_split(X_fair, y_fair, test_size = 0.3, random_state = 1)



In [ ]:
# combine fairbanks and anchorage
X_train = np.vstack((X_train_anch, X_train_fair))
X_test = np.vstack((X_test_anch, X_test_fair))
y_train = np.vstack((y_train_anch, y_train_fair))
y_test = np.vstack((y_test_anch, y_test_fair))

In [ ]:
with open(path + 'data_sampling/equal_anch_fair/x_train_upsample_fairbanks.npy', 'wb') as f:
    np.save(f, X_train)

    
with open(path + 'data_sampling/equal_anch_fair/x_test_upsample_fairbanks.npy', 'wb') as f:
    np.save(f, X_test)
    
    
with open(path + 'data_sampling/equal_anch_fair/y_train_upsample_fairbanks.npy', 'wb') as f:
    np.save(f, y_train)
    
with open(path + 'data_sampling/equal_anch_fair/y_test_upsample_fairbanks.npy', 'wb') as f:
    np.save(f, y_test)

# Building age

In [ ]:
joined_data = pd.read_csv(path + 'data_btu_railbelt.csv')
joined_data = joined_data.drop(columns =['Unnamed: 0'])

In [ ]:
# bin age
joined_data['year_rounded'] = abs(joined_data['age'].round() - 2020)

In [ ]:
# change FD cols to be positive
joined_data['FD_1981_2010'] = abs(joined_data['FD_1981_2010'])
joined_data['FD_2000'] = abs(joined_data['FD_2000'])
joined_data['FD_2010'] = abs(joined_data['FD_2010'])
joined_data['FD_90'] = abs(joined_data['FD_90'])
joined_data['FD_80'] = abs(joined_data['FD_80'])

In [ ]:
joined_data.head()

In [ ]:
joined_data_nonmissing = joined_data.dropna()

In [ ]:
joined_data_nonmissing['year_rounded'].unique()

In [ ]:
years_df_dict = {}
for year in joined_data_nonmissing['year_rounded'].unique():
    years_df_dict[year] = joined_data_nonmissing[joined_data_nonmissing['year_rounded'] == year]

In [ ]:
size_of_bins = []
for key in years_df_dict.keys():
    size_of_bins.append(len(years_df_dict[key]))

In [ ]:
print(np.mean(size_of_bins))

In [ ]:
max_count = 0
for year in years_df_dict.keys():
    #print('year: ', year)
    #print(len(years_df_dict[year]))
    
    if len(years_df_dict[year])> max_count:
        max_count = len(years_df_dict[year])

In [ ]:
max_count

In [ ]:
# up and down sample to 5000
num_samples = 2200 #5000
sampled_dfs = {}
for year in years_df_dict.keys():
    current_df = years_df_dict[year]
    num_buildings = len(current_df)
    new_df = pd.DataFrame()
    
    if num_buildings > num_samples:
        # downsample
        new_df = new_df.append(current_df.sample(n=num_samples)) 
    elif num_buildings < num_samples:
        # upsample
        new_df = new_df.append(current_df)
        new_df = new_df.append(current_df.sample(n=num_samples - len(current_df), replace=True))
    else:
        new_df = current_df
    
    sampled_dfs[year] = new_df
    

In [ ]:
# Create numpy arrays for scikit learn 
features_dropped = ['annual_btu_persqft', 'annual_btu', 'age', 'osm_id', 'zip_code', 'zip_group', 'year_rounded']
# features_kept = combined_fair_anch.drop(features_dropped, axis=1).columns

X_list = []
y_list = []
for year in sampled_dfs.keys():
    df = sampled_dfs[year]
    X = np.array(df.drop(features_dropped, axis=1))
    y = np.array(df['annual_btu_persqft'])
    y = y.reshape([y.shape[0],1])
    
    X_list.append(X)
    y_list.append(y)
    

In [ ]:
sampled_dfs[0].columns

In [ ]:
# standardize data
scaler = StandardScaler()
scaler.fit(np.vstack((X_list)))
X_list = [scaler.transform(x_elem) for x_elem in X_list]


scaler = StandardScaler()
scaler.fit(np.vstack((y_list)))
y_list = [scaler.transform(y_elem) for y_elem in y_list]

In [ ]:
X_train_list = []
X_test_list = []
y_train_list = []
y_test_list = []
for i in range(len(X_list)):
    X = X_list[i]
    y = y_list[i]
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 1)
    
    X_train_list.append(X_train)
    X_test_list.append(X_test)
    y_train_list.append(y_train)
    y_test_list.append(y_test)


In [ ]:
# combine ages
X_train = np.vstack((X_train_list))
X_test = np.vstack((X_test_list))
y_train = np.vstack((y_train_list))
y_test = np.vstack((y_test_list))

In [ ]:
with open(path + 'data_sampling/equal_age/x_train_2200.npy', 'wb') as f:
    np.save(f, X_train)

    
with open(path + 'data_sampling/equal_age/x_test_2200.npy', 'wb') as f:
    np.save(f, X_test)
    
    
with open(path + 'data_sampling/equal_age/y_train_2200.npy', 'wb') as f:
    np.save(f, y_train)
    
with open(path + 'data_sampling/equal_age/y_test.npy_2200', 'wb') as f:
    np.save(f, y_test)

# equal sq ft-- unsure about how to choose the bins here

In [ ]:
joined_data = pd.read_csv(path + 'data_btu_railbelt.csv')
joined_data = joined_data.drop(columns =['Unnamed: 0'])

In [ ]:
joined_data_nonmissing = joined_data.dropna()

In [ ]:
pd.cut(np.array([1, 7, 5, 4, 6, 3]), 3)

In [ ]:
joined_data_nonmissing['sq_ft_bin'] = pd.qcut(joined_data_nonmissing['areasq_ft'], 50, labels = np.arange(50))


In [ ]:
joined_data_nonmissing.head()

In [ ]:
for i in range(50):
    print(len(joined_data_nonmissing[joined_data_nonmissing['sq_ft_bin'] == i]))
    print(joined_data_nonmissing[joined_data_nonmissing['sq_ft_bin'] ==i]['sq_ft_bin'].max())
    

In [ ]:
# # bin sq ft
# joined_data['year_rounded'] = abs(joined_data['age'].round() - 2020)

In [ ]:
# # change FD cols to be positive
# joined_data['FD_1981_2010'] = abs(joined_data['FD_1981_2010'])
# joined_data['FD_2000'] = abs(joined_data['FD_2000'])
# joined_data['FD_2010'] = abs(joined_data['FD_2010'])
# joined_data['FD_90'] = abs(joined_data['FD_90'])
# joined_data['FD_80'] = abs(joined_data['FD_80'])

In [ ]:
# joined_data_nonmissing['year_rounded'].unique()

In [ ]:
# years_df_dict = {}
# for year in joined_data_nonmissing['year_rounded'].unique():
#     years_df_dict[year] = joined_data_nonmissing[joined_data_nonmissing['year_rounded'] == year]

In [ ]:
# max_count = 0
# for year in years_df_dict.keys():
#     #print('year: ', year)
#     #print(len(years_df_dict[year]))
    
#     if len(years_df_dict[year])> max_count:
#         max_count = len(years_df_dict[year])

In [ ]:
# # up and down sample to 5000
# num_samples = 5000
# sampled_dfs = {}
# for year in years_df_dict.keys():
#     current_df = years_df_dict[year]
#     num_buildings = len(current_df)
#     new_df = pd.DataFrame()
    
#     if num_buildings > num_samples:
#         # downsample
#         new_df = new_df.append(current_df.sample(n=num_samples)) 
#     elif num_buildings < num_samples:
#         # upsample
#         new_df = new_df.append(current_df)
#         new_df = new_df.append(current_df.sample(n=num_samples - len(current_df), replace=True))
#     else:
#         new_df = current_df
    
#     sampled_dfs[year] = new_df
    

In [ ]:
# # Create numpy arrays for scikit learn 
# features_dropped = ['annual_btu_persqft', 'annual_btu', 'age', 'osm_id', 'zip_code', 'zip_group', 'sq_ft_bin']
# # features_kept = combined_fair_anch.drop(features_dropped, axis=1).columns

# X_list = []
# y_list = []
# for year in sampled_dfs.keys():
#     df = sampled_dfs[year]
#     X = np.array(df.drop(features_dropped, axis=1))
#     y = np.array(df['annual_btu_persqft'])
#     y = y.reshape([y.shape[0],1])
    
#     X_list.append(X)
#     y_list.append(y)
    

In [ ]:
# # standardize data
# scaler = StandardScaler()
# scaler.fit(np.vstack((X_list)))
# X_list = [scaler.transform(x_elem) for x_elem in X_list]


# scaler = StandardScaler()
# scaler.fit(np.vstack((y_list)))
# y_list = [scaler.transform(y_elem) for y_elem in y_list]

In [ ]:
# X_train_list = []
# X_test_list = []
# y_train_list = []
# y_test_list = []
# for i in range(len(X_list)):
#     X = X_list[i]
#     y = y_list[i]
    
#     X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 1)
    
#     X_train_list.append(X_train)
#     X_test_list.append(X_test)
#     y_train_list.append(y_train)
#     y_test_list.append(y_test)


In [ ]:
# # combine ages
# X_train = np.vstack((X_train_list))
# X_test = np.vstack((X_test_list))
# y_train = np.vstack((y_train_list))
# y_test = np.vstack((y_test_list))

In [ ]:
# with open(path + 'data_sampling/equal_age/x_train.npy', 'wb') as f:
#     np.save(f, X_train)

    
# with open(path + 'data_sampling/equal_age/x_test.npy', 'wb') as f:
#     np.save(f, X_test)
    
    
# with open(path + 'data_sampling/equal_age/y_train.npy', 'wb') as f:
#     np.save(f, y_train)
    
# with open(path + 'data_sampling/equal_age/y_test.npy', 'wb') as f:
#     np.save(f, y_test)

# all regressions

## Linear Regression

In [ ]:
# Linear Regression
lm = LinearRegression()
lm.fit(X_train,y_train)

y_pred_lm = lm.predict(X_test) # predicting using train dataset

coeffs = lm.coef_
print(coeffs)

In [ ]:
mse_lm = mean_squared_error(y_test, y_pred_lm)
print("MSE model_linear:", mse_lm)

## Ridge Regression

In [ ]:
# Ridge Regression
ridge = Ridge(alpha= 0.5)
ridge.fit(X_train, y_train)

y_pred_ridge = ridge.predict(X_test)
coeffs = ridge.coef_
print(coeffs)

In [ ]:
mse_ridge = mean_squared_error(y_test, y_pred_ridge)
print("MSE model_ridge:", mse_ridge)

## Ridge Regression with Cross Validation--?

In [ ]:
# Ridge Regression with CV and with polynomial degree 2 features

# Polynomial Fit

poly_train = PolynomialFeatures(2)
X_train_poly = poly_train.fit_transform(X_train)

poly_test = PolynomialFeatures(2)
X_test_poly = poly_test.fit_transform(X_test)

ridgeCV_poly = RidgeCV(cv=10).fit(X_train_poly,y_train)
y_pred_ridgeCV_poly = ridgeCV_poly.predict(X_test_poly)


In [ ]:
mse_ridgeCV_poly = mean_squared_error(y_test, y_pred_ridgeCV_poly)
print("MSE model_ridgeCV_poly:", mse_ridgeCV_poly)

## Decision tree

In [ ]:
# Decision tree
dt = DecisionTreeRegressor(random_state=3).fit(X_train, y_train)
y_pred_dt = dt.predict(X_test)

In [ ]:
mse_dt = mean_squared_error(y_test, y_pred_dt)
print("MSE model_dt:", mse_dt)

## Random Forest Regressor

In [ ]:
# Random Forest Regressor
rf = RandomForestRegressor(max_depth=15, random_state=3)
rf.fit(X_train, y_train.reshape(len(y_train)))

y_pred_rf = rf.predict(X_test)


In [ ]:
mse_rf = mean_squared_error(y_test, y_pred_rf)
print("MSE model_rf:", mse_rf)